In [1]:
import numpy as np
import opendssdirect as dss
from utils.device.Inverter import Inverter
from utils.controller.AdaptiveInvController import AdaptiveInvController
from utils.controller.FixedInvController import FixedInvController
import matplotlib.pyplot as plt
from math import tan,acos
import copy
import pandas as pd
import time

%matplotlib inline

# import numpy as np
# from matplotlib.mlab import PCA
from sklearn.decomposition import PCA

import scipy.linalg as spla


#######################################################
#######################################################
###Global variable initialization and error checking###
#######################################################
#######################################################
Sbase=1
LoadScalingFactor = 3
GenerationScalingFactor = 5
SlackBusVoltage = 1.00
NoiseMultiplyer= 1
#Set simulation analysis period - the simulation is from StartTime to EndTime
StartTime = 40600
EndTime = StartTime + 640
EndTime += 1 # creating a list, last element does not count, so we increase EndTime by 1
#Set hack parameters
TimeStepOfHack = 160
PercentHacked = np.array([0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0])

#Set initial VBP parameters for uncompromised inverters
VBP_normal=np.array([0.97, 1, 1, 1.03, 1.06])
VBP_attack=np.array([0.997, 1, 1, 1.003, 1.007])

#Set delays for each node
Delay_VBPCurveShift = (30+2*np.random.randn(31)).astype(int)
#Delay_VBPCurveShift = (10+2*np.random.randn(31)).astype(int)
lpf_meas_vector = (1+0.2*np.random.randn(31))
lpf_output_vector= (0.1+0.015*np.random.randn(31))

#Set observer voltage threshold
ThreshHold_vqvp = 0.06
adaptive_gain=800


power_factor=0.9
pf_converted=tan(acos(power_factor))
# Number_of_Inverters = 13 #even feeder is 34Bus, we only have 13 inverters


#Error checking of the global variable -- TODO: add error handling here!
if EndTime < StartTime or EndTime < 0 or StartTime < 0:
    print('Setup Simulation Times Inappropriately.')
if NoiseMultiplyer < 0:
    print('Setup Noise Multiplyer Correctly.')
    
dss.run_command('Redirect feeder/03node_multiphase_balanced.dss')
# dss.run_command('Redirect feeder/feeder34_B_NR/network_05node_3phase_radial.dss')

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('')
print(TotalLoads)
# dir(dss.Loads)

# (dss.Loads.AllNames())

# [x for x in dss.Circuit.AllBusNames() if 'bus' in x ]

# dss.Circuit.AllBusNames()

# dir(dss.Loads.AllNames)
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

print(dss.Bus.kVBase())

for k1 in range(len(dss.Circuit.AllBusNames())):
    if dss.Circuit.AllBusNames()[k1] == 'sourcebus':
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
#         print(dss.Bus.kVBase())
        tempVoltage = dss.Bus.VMagAngle()
        tempVoltage = tempVoltage
    #     print(tempVoltage)
#         print(tempVoltage[::2])
#         print(tempVoltage[0]*np.sqrt(3)/1)
        
        print(dss.Circuit.AllBusNames()[k1])
        print('BASE - LN')
        print(dss.Bus.kVBase())
        print('VMAGANGLE - LN POLAR')
        print(dss.Bus.VMagAngle())
        print('puVMAGANGLE - LN POLAR')
        print(dss.Bus.puVmagAngle())
        print('VOTLAGES - LN CARTESIAN')
        print(dss.Bus.Voltages())
        print('puVOTLAGES - LN CARTESIAN')
        print(dss.Bus.PuVoltage())
        print('VLL - LL CARTESIAN')
        print(dss.Bus.VLL())
        print('puVLL - LL CARTESIAN')
        print(dss.Bus.puVLL())
        
        tempvoltage = dss.Bus.Voltages()
        
        Vab = tempvoltage[0]
        
print('~~~~~')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    
#     print('puVLL - LL CARTESIAN')
#     print(dss.Bus.puVLL())
#     tempVoltage = np.array(dss.Bus.puVLL())#.astype('float')
#     tempMag = np.array([np.sqrt(tempVoltage[0]**2 + tempVoltage[1]**2), \
#                         np.sqrt(tempVoltage[2]**2 + tempVoltage[3]**2), \
#                         np.sqrt(tempVoltage[4]**2 + tempVoltage[5]**2)])
#     print(tempMag)
#     tempAngle = np.array([np.angle(tempVoltage[0] + 1j*tempVoltage[1], deg=True), \
#                          np.angle(tempVoltage[2] + 1j*tempVoltage[3], deg=True)+360, \
#                          np.angle(tempVoltage[4] + 1j*tempVoltage[5], deg=True)])
#     print(tempAngle)
    print('')

    
# dir(dss.Bus)

ModuleNotFoundError: No module named 'opendssdirect'

In [2]:
print(dss.RegControls.AllNames())

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())
    

# dss.RegControls.Name('reg1')
# dss.RegControls.Name()

# for k1 in range(len(dss.RegControls.AllNames())):
#     print(dss.RegControls.AllNames()[k1])
#     dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
#     dss.RegControls.TapNumber(0)    
#     print(dss.RegControls.TapNumber())


['reg1']
reg1
6


In [3]:
dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('~~~~~')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    print('')

print('')
print(dss.RegControls.AllNames())

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
~~~~~
sourcebus
BASE - LN
132.79056191361394
VMAGANGLE - LN POLAR
[132789.72056101917, -0.0005987280265653524, 132789.65752056905, -120.00059282719847, 132789.6561900636, 119.99940988636436]
puVMAGANGLE - LN POLAR
[0.9999936640632991, -0.0005987280265653524, 0.999993189327375, -120.00059282719847, 0.9999931793077966, 119.99940988636436]

799
BASE - LN
2.7712812921102037
VMAGANGLE - LN POLAR
[2662.7892943820307, -4.112705313444468, 2653.1320522265723, -124.08092212015843, 2653.132022745855, 115.91908051186827]
puVMAGANGLE - LN POLAR
[0.9608513224416995, -4.112705313444468, 0.9573665653428974, -124.08092212015843, 0.9573665547049598, 115.91908051186827]

799r
BASE - LN
2.7712812921102037
VMAGANGLE - LN POLAR
[2762.531714995935, -4.114160088989235, 2752.5078273741483, -124.08230393197269, 2752.5077967865427, 115.9176987000275]
puVMAGANGLE - LN POLAR
[0.9968427683111135, -4.114160088989235, 0.9932257094256354

In [4]:
for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    dss.RegControls.Reset()    
    print(dss.RegControls.TapNumber())

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('~~~~~')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    print('')

print('')
print(dss.RegControls.AllNames())

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())

reg1
6
Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
~~~~~
sourcebus
BASE - LN
132.79056191361394
VMAGANGLE - LN POLAR
[132789.72056101917, -0.0005987280265653524, 132789.65752056905, -120.00059282719847, 132789.6561900636, 119.99940988636436]
puVMAGANGLE - LN POLAR
[0.9999936640632991, -0.0005987280265653524, 0.999993189327375, -120.00059282719847, 0.9999931793077966, 119.99940988636436]

799
BASE - LN
2.7712812921102037
VMAGANGLE - LN POLAR
[2662.7892943820307, -4.112705313444468, 2653.1320522265723, -124.08092212015843, 2653.132022745855, 115.91908051186827]
puVMAGANGLE - LN POLAR
[0.9608513224416995, -4.112705313444468, 0.9573665653428974, -124.08092212015843, 0.9573665547049598, 115.91908051186827]

799r
BASE - LN
2.7712812921102037
VMAGANGLE - LN POLAR
[2762.531714995935, -4.114160088989235, 2752.5078273741483, -124.08230393197269, 2752.5077967865427, 115.9176987000275]
puVMAGANGLE - LN POLAR
[0.9968427683111135, -4.114160088989235, 0.993225709

In [5]:
dss.run_command('Redirect feeder/03node_multiphase_balanced.dss')

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('~~~~~')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())
    
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print(dss.Bus.puVmagAngle()[::2])
#     print('')

dss.run_command('Redirect feeder/03node_multiphase_balanced.dss')


# print(dss.Loads.AllNames())
dss.Loads.Name('s701a')
dss.Loads.kvar(-500)

dss.Loads.Name('s702a')
dss.Loads.kvar(-500)

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('~~~~~')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())
    
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print(dss.Bus.puVmagAngle()[::2])
#     print('')

dss.run_command('Redirect feeder/03node_multiphase_balanced.dss')

dss.Loads.Name('s701a')
dss.Loads.kvar(500)

dss.Loads.Name('s702a')
dss.Loads.kvar(500)

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    dss.RegControls.Reset()

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('~~~~~')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    
    print(dss.RegControls.TapNumber())
    
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print(dss.Bus.puVmagAngle()[::2])
#     print('')

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
~~~~~
reg1
6
sourcebus
[0.9999936639245567, 0.9999931888075513, 0.9999931787385393]
799
[0.9608505591064769, 0.9573632068692834, 0.9573631961206244]
799r
[0.9968419756566543, 0.9932222221973943, 0.9932222110451171]
701
[0.9846287675752631, 0.9706531176672704, 0.970653106192043]
702
[0.9706938063844576, 0.9564953908064358, 0.9564953791471135]
Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
~~~~~
reg1
-10
sourcebus
[1.0000081913379788, 0.9999948435125198, 0.9999947904129678]
799
[1.0640006780373392, 0.9647542720464553, 0.9647542210564314]
799r
[0.9975067608455582, 0.9044253822452413, 0.9044253344439697]
701
[1.0000282121379156, 0.8835815854423374, 0.8835815388810988]
702
[1.0247199326610066, 0.8706449308513012, 0.8706448850550368]
reg1
Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
~~~~~
reg1
16
sourcebus
[0.9999810113944185, 0.99999283969